<a href="https://colab.research.google.com/github/Stekosan/merrer/blob/main/Copy_of_SparkNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let's set up SparkNLP.

In [20]:
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2024-07-27 07:31:15--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 3.86.22.73
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|3.86.22.73|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2024-07-27 07:31:15--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1191 (1.2K) [text/plain]
Saving to: ‘STDOUT’

Installing PySpark 3.2.3 and Spark NLP 5.4.1
-                   100%[===================>]   1.16K  --.-KB/s    in 0s      

2024-07-27 07:31:15 (46.8 MB/s) - writ

In [21]:
import sparknlp
spark = sparknlp.start()

from sparknlp.pretrained import PretrainedPipeline

In [22]:
pipeline = PretrainedPipeline("explain_document_ml")

explain_document_ml download started this may take some time.
Approx size to download 9 MB
[OK!]


We can use some recent headlines.

In [23]:
hls = [ # was headlines
		"She",
		"He",
		"her",
		"him",
		"hers",
		"his"
	]

Let's use SparkNLP to analyze these headlines.

In [24]:
# Use dataframes, or...
# data = spark.createDataFrame(hls).toDF("text")
# dfs = pipeline.transform(data)
# ... use list comprehension
dfs = [pipeline.annotate(hl) for hl in hls] # I don't know how to use dataframes

In [25]:
# its big
dfs

[{'document': ['She'],
  'spell': ['She'],
  'pos': ['PRP'],
  'lemmas': ['She'],
  'token': ['She'],
  'stems': ['she'],
  'sentence': ['She']},
 {'document': ['He'],
  'spell': ['He'],
  'pos': ['PRP'],
  'lemmas': ['He'],
  'token': ['He'],
  'stems': ['he'],
  'sentence': ['He']},
 {'document': ['her'],
  'spell': ['her'],
  'pos': ['PRP$'],
  'lemmas': ['she'],
  'token': ['her'],
  'stems': ['her'],
  'sentence': ['her']},
 {'document': ['him'],
  'spell': ['him'],
  'pos': ['PRP'],
  'lemmas': ['he'],
  'token': ['him'],
  'stems': ['him'],
  'sentence': ['him']},
 {'document': ['hers'],
  'spell': ['hers'],
  'pos': ['NNS'],
  'lemmas': ['hers'],
  'token': ['hers'],
  'stems': ['her'],
  'sentence': ['hers']},
 {'document': ['his'],
  'spell': ['his'],
  'pos': ['PRP$'],
  'lemmas': ['he'],
  'token': ['his'],
  'stems': ['hi'],
  'sentence': ['his']}]

Let's say we want to fuse part-of-speech tags to words, to make word differentiation easier.

In [26]:
# Extract words and parts-of-speech
tok_tag = [(df['token'],df['pos']) for df in dfs]

In [27]:
# Still big
tok_tag

[(['She'], ['PRP']),
 (['He'], ['PRP']),
 (['her'], ['PRP$']),
 (['him'], ['PRP']),
 (['hers'], ['NNS']),
 (['his'], ['PRP$'])]

In [28]:
# fuse pos to word
zips = [list(zip(tt[0], tt[1])) for tt in tok_tag]

In [29]:
# not too big
zips

[[('She', 'PRP')],
 [('He', 'PRP')],
 [('her', 'PRP$')],
 [('him', 'PRP')],
 [('hers', 'NNS')],
 [('his', 'PRP$')]]

In [30]:
tagged = [" ".join(["".join(word) for word in hl]) for hl in zips]

In [31]:
tagged

['ShePRP', 'HePRP', 'herPRP$', 'himPRP', 'hersNNS', 'hisPRP$']

What about ebooks?

In [54]:
!curl "https://raw.githubusercontent.com/Stekosan/merrer/main/sparknlp/Diaries_of_Court_Ladies_of_Old_Japan.txt" -o diary.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  325k  100  325k    0     0   582k      0 --:--:-- --:--:-- --:--:--  581k


In [55]:
diary = open('diary.txt').read()

In [56]:
print(diary[:1000])

﻿*** START OF THE PROJECT GUTENBERG EBOOK DIARIES OF COURT LADIES OF OLD JAPAN ***

DIARIES OF

COURT LADIES OF OLD JAPAN

TRANSLATED BY

ANNIE SHEPLEY OMORI

AND

KOCHI DOI

_Professor in the Imperial University, Tokio_

WITH AN INTRODUCTION BY

AMY LOWELL


_And with Illustrations_


BOSTON AND NEW YORK

HOUGHTON MIFFLIN COMPANY

The Riverside Press Cambridge

1920



[Illustration: COURT LADY'S FULL DRESS IN THE HEIAN PERIOD]

(For explanation see List of Illustrations)




TRANSLATORS' NOTE


The poems in the text, slight and occasional as they are, depending
often for their charm on plays upon words of two meanings, or on
the suggestions conveyed to the Japanese mind by a single word,
have presented problems of great difficulty to the translators, not
perfectly overcome.

Izumi Shikibu's Diary is written with extreme delicacy of treatment.
English words and thought seem too downright a medium into which to
render these evanescent, half-expressed sentences and poems--vague as
the m

In [57]:
pipeline.annotate(diary[:100])['pos']

['NN',
 'NN',
 'NNP',
 'IN',
 'DT',
 'NNP',
 'NNP',
 'NNP',
 'NNP',
 'IN',
 'NNP',
 'NNP',
 'IN',
 'NNP',
 'NNP',
 'NN',
 'NNP',
 'IN',
 'PRP$']

Previously with ebooks, we conducted word counts. We can do that here as well, with Spark.

In [58]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("demo").getOrCreate()

In [59]:
# change 'austen' variable from a string to a spark object
diary = spark.sparkContext.textFile("diary.txt")

counts = (
    diary.flatMap(lambda line: line.split(" "))
    .map(lambda word: (word, 1))
    .reduceByKey(lambda a, b: a + b)
)

In [60]:
counts.collect()[:100]

[('***', 4),
 ('OF', 23),
 ('GUTENBERG', 2),
 ('DIARIES', 5),
 ('COURT', 8),
 ('LADIES', 4),
 ('OLD', 7),
 ('JAPAN', 4),
 ('', 7244),
 ('TRANSLATED', 1),
 ('ANNIE', 1),
 ('SHEPLEY', 1),
 ('OMORI', 1),
 ('KOCHI', 1),
 ('_Professor', 1),
 ('in', 922),
 ('Imperial', 17),
 ('University,', 1),
 ('Tokio_', 1),
 ('INTRODUCTION', 3),
 ('LOWELL', 3),
 ('Illustrations_', 1),
 ('NEW', 1),
 ('YORK', 1),
 ('HOUGHTON', 1),
 ('MIFFLIN', 1),
 ('The', 506),
 ('Riverside', 1),
 ('Cambridge', 1),
 ('1920', 1),
 ("LADY'S", 2),
 ('FULL', 2),
 ('HEIAN', 2),
 ('List', 2),
 ('of', 1803),
 ('NOTE', 1),
 ('poems', 28),
 ('text,', 2),
 ('occasional', 5),
 ('as', 247),
 ('are,', 3),
 ('depending', 1),
 ('plays', 1),
 ('upon', 35),
 ('two', 51),
 ('conveyed', 1),
 ('mind', 46),
 ('single', 3),
 ('have', 213),
 ('presented', 17),
 ('difficulty', 3),
 ('perfectly', 3),
 ('overcome.', 1),
 ('is', 547),
 ('extreme', 5),
 ('delicacy', 2),
 ('treatment.', 1),
 ('English', 2),
 ('thought', 84),
 ('downright', 1),
 ('medi